<img src='https://www.icos-cp.eu/sites/default/files/2017-11/ICOS_CP_logo.png' width=400 align=right>

# ICOS Carbon Portal Python Libraries: icoscp_core

This example uses a foundational library called `icoscp_core` which can be used to access time-series ICOS data that are <i>previewable</i> in the ICOS Data Portal. "Previewable" means that it is possible to visualize the data variables in the preview plot. The library can also be used to access (meta-)data from [ICOS Cities](https://citydata.icos-cp.eu/portal/) and [SITES](https://data.fieldsites.se/portal/) data repositories. 

Documentation of the library, including information on running it locally, can be found on [PyPI.org](https://pypi.org/project/icoscp_core/).


# Example: Explore ICOS stations and list their data


## Import the library

In [ ]:
from icoscp_core.icos import meta, station_class_lookup

## Get help

In [ ]:
import icoscp_core
help(meta.list_stations)
#help(icoscp_core.queries.stationlist.StationLite)

## List of stations
Get a list of all ICOS stations as a pandas dataframe table with basic information. Only final certified ICOS stations are listed.
<br>Good to know:
- uri => landing page URI and stable metadata id for the station at the ICOS Carbon Portal
- id  => ICOS-internal station id
- elevation => meter above sea level
- type_uri => URI representing the type of station, last segment of the URI: ES=EcoSystem, AS=Atmosphere, OS=Ocean


In [ ]:
import pandas as pd

icoslike_stations = meta.list_stations()

icos_stations = [
    s for s in icoslike_stations
    # only stations with ICOS station class are proper ICOS stations
    if station_class_lookup().get(s.uri)
]

pd.DataFrame(icos_stations)

## Map of stations
Generate a folium map of stations. Each marker in the map represents a station and contains station related information.

In [ ]:
import folium

icon_lookup = {}
for st_class in ["AS", "ES", "OS"]:
    st_class_uri = f"http://meta.icos-cp.eu/ontologies/cpmeta/{st_class}"
    icon_uri = f"https://static.icos-cp.eu/share/stations/icons/{st_class.lower()}.png"
    icon_lookup[st_class_uri] = icon_uri

myMap = folium.Map(location=[53, 0], zoom_start=4)

for s in icos_stations:

    if s.geo_json:
        folium.GeoJson(
            data=s.geo_json,
            tooltip=s.label,
            marker=folium.CircleMarker()
        ).add_to(myMap)

    if s.lat:
        icon_url = icon_lookup.get(s.type_uri)
        folium.Marker(
            location=[s.lat, s.lon],
            tooltip=s.label,
            icon=folium.CustomIcon(icon_url, (23, 28)) if icon_url is not None else None
        ).add_to(myMap)
myMap

## Station meta data

### Fetch and use station metadata

In [ ]:
# pick a station from the list of stations above, use its URI to identify it
zep_uri = "http://meta.icos-cp.eu/resources/stations/AS_ZEP"
zep = meta.get_station_meta(zep_uri)

In [ ]:
zep.id

In [ ]:
zep.org.name

In [ ]:
zep.org.self.uri

In [ ]:
zep.location

In [ ]:
zep.pictures

In [ ]:
zep.specificInfo

In [ ]:
zep.staff

### Station metadata reference

Available (nested) metadata properties can be discovered with Tab-completion after "."

For a more principled discovery of the metadata properties, one can examine Python documentation for respective types as follows.

In [ ]:
from icoscp_core import metacore
from icoscp_core.metaclient import Station
help(Station)
#help(metacore.Organization)
#help(metacore.UriResource)
#metacore.URI # alias for str
#help(metacore.Position)
#help(icoscp_core.rolemeta.Membership)
#help(icoscp_core.rolemeta.RoleDetails)
#help(metacore.Funding)
#metacore.StationSpecifics
#help(metacore.AtcStationSpecifics)
#metacore.IcosStationClass

## Data associated with a station
List all L2 data objects associated with a specific station.

In [ ]:
# first discover all L2 data types
l2_data_types = [dt for dt in meta.list_datatypes() if dt.data_level == 2]
data_objs = meta.list_data_objects(datatype=l2_data_types, station=zep_uri)
pd.DataFrame(data_objs)